In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import dask
import os
import glob
import sys

In [16]:
client.close()
cluster.close()

In [2]:
import dask.distributed as dd
import dask
import dask.array as da
# Create a Dask cluster
print("Starting parallel computing...")
cluster = dd.LocalCluster(n_workers=8, dashboard_address=':22722')

# Connect to the cluster
client = dd.Client(cluster)
client

Starting parallel computing...


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:22722/status,
Dashboard: http://127.0.0.1:22722/status,Workers: 8
Total threads: 96,Total memory: 187.55 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35861,Workers: 8
Dashboard: http://127.0.0.1:22722/status,Total threads: 96
Started: Just now,Total memory: 187.55 GiB
Comm: tcp://127.0.0.1:46151,Total threads: 12
Dashboard: http://127.0.0.1:35523/status,Memory: 23.44 GiB
Nanny: tcp://127.0.0.1:34407,


In [3]:
# SFC
source_dir = 'CERRA_complete'
var_names = ['t2m','si10','skt']
for var in var_names:
    files = sorted(glob.glob(f'{source_dir}/{var}/*.nc'))
    ds = xr.open_mfdataset(files,concat_dim='valid_time',parallel=True,combine='nested')
    ds.to_netcdf(f'{source_dir}/{var}.nc')

In [12]:
# PRES
source_dir = 'CERRA_complete'
var = 'PRES'
files = sorted(glob.glob(f'{source_dir}/{var}/*.nc'))
ds = xr.open_mfdataset(files,concat_dim='valid_time',parallel=True,combine='nested')
ds.to_netcdf(f'{source_dir}/{var}.nc')

### Combining individual variables

In [13]:
def compute_wind_speed(ds1,ds2, par_name):
    ds = (ds1**2+ds2**2)**0.5
    ds = ds.rename(par_name)
    return ds
def compute_alpha(dataset, par_name1,par_name2, par_name):
    ds = np.log(dataset[par_name2]/dataset[par_name1])/np.log(100/10)
    ds = ds.rename(par_name)
    return ds
def compute_gradient(dataset, par_name1,par_name2, par_name):
    ds = dataset[par_name2]-dataset[par_name1]
    ds = ds.rename(par_name)
    return ds

In [14]:
combined_dataset = xr.Dataset()

var = 'si10'
ds = xr.open_dataset(f'{source_dir}/{var}.nc')
ds = ds[var].drop('heightAboveGround')
ds = ds.rename('10ws')
combined_dataset = xr.merge([combined_dataset, ds])

# 100m wind speed from CERRA height level
files = sorted(glob.glob(f'CERRA_height_level/*.nc'))
ds = xr.open_mfdataset(files,concat_dim='time',parallel=True,combine='nested')
# Rename the coordinate from 'obs' to 'location'
ds = ds.rename({'obs': 'location'})
ds = ds.rename({'lat': 'latitude'})
ds = ds.rename({'lon': 'longitude'})
# Extract data at every 3rd hour
ds = ds.sel(time=ds.time.dt.hour % 3 == 0)
# Rename the variable to 'ws100'
ds = ds.rename({'data': '100ws'})
# selecting 100m level
ds = ds.sel(heightAboveGround=100).drop('heightAboveGround')
# Rename the coordinate from 'time' to 'valid_time'
ds = ds.rename({'time': 'valid_time'})
combined_dataset = xr.merge([combined_dataset, ds],compat='override')

# 975 ws
ds1 = xr.open_dataset(f'{source_dir}/PRES.nc')['u'].sel(isobaricInhPa=975).drop('isobaricInhPa')
ds2 = xr.open_dataset(f'{source_dir}/PRES.nc')['v'].sel(isobaricInhPa=975).drop('isobaricInhPa')
ds = compute_wind_speed(ds1,ds2, '975ws')
combined_dataset = xr.merge([combined_dataset, ds])

# 950 ws
ds1 = xr.open_dataset(f'{source_dir}/PRES.nc')['u'].sel(isobaricInhPa=950).drop('isobaricInhPa')
ds2 = xr.open_dataset(f'{source_dir}/PRES.nc')['v'].sel(isobaricInhPa=950).drop('isobaricInhPa')
ds = compute_wind_speed(ds1,ds2, '950ws')
combined_dataset = xr.merge([combined_dataset, ds])

var = 't2m'
ds = xr.open_dataset(f'{source_dir}/{var}.nc')
ds = ds[var].drop('heightAboveGround')
combined_dataset = xr.merge([combined_dataset, ds])

var = 'skt'
ds = xr.open_dataset(f'{source_dir}/{var}.nc')
ds = ds[var].drop('surface')
combined_dataset = xr.merge([combined_dataset, ds])

# 975 t
ds1 = xr.open_dataset(f'{source_dir}/PRES.nc')['t'].sel(isobaricInhPa=975).drop('isobaricInhPa')
ds = compute_wind_speed(ds1,ds2, 't_975')
combined_dataset = xr.merge([combined_dataset, ds])

# 950 t
ds1 = xr.open_dataset(f'{source_dir}/PRES.nc')['t'].sel(isobaricInhPa=950).drop('isobaricInhPa')
ds = compute_wind_speed(ds1,ds2, 't_950')
combined_dataset = xr.merge([combined_dataset, ds])

# === At present, the SURF variables are having only data during '2000-01-01T00','2019-12-31T21. So, we trim accordingly
combined_dataset = combined_dataset.sel(valid_time=slice('2000-01-01T00','2019-12-31T21'))

# === derived parameters === #
# --- 100 alpha ---#
ds = compute_alpha(combined_dataset,'10ws','100ws','100alpha')
combined_dataset = xr.merge([combined_dataset, ds])

# --- 975 wind gradient ---#
ds = compute_gradient(combined_dataset,'100ws','975ws','975wsgrad')
combined_dataset = xr.merge([combined_dataset, ds])

# --- 950 wind gradient ---#
ds = compute_gradient(combined_dataset,'975ws','950ws','950wsgrad')
combined_dataset = xr.merge([combined_dataset, ds]) 

# --- 2m temperature gradient ---#
ds = compute_gradient(combined_dataset,'skt','t2m','2mtempgrad')
combined_dataset = xr.merge([combined_dataset, ds]) 

# --- 950 temperature gradient ---#
ds = compute_gradient(combined_dataset,'t_975','t_950','950tempgrad')
combined_dataset = xr.merge([combined_dataset, ds]) 

# --- 975 temperature gradient ---#
ds = compute_gradient(combined_dataset,'t2m','t_975','975tempgrad')
combined_dataset = xr.merge([combined_dataset, ds]) 

combined_dataset = combined_dataset.drop(['time','step']).rename({'valid_time': 'time'})
combined_dataset


<xarray.Dataset>
Dimensions:      (time: 58440, location: 11)
Coordinates:
  * time         (time) datetime64[ns] 2000-01-01 ... 2019-12-31T21:00:00
    latitude     (location) float64 54.0 51.99 56.42 51.73 ... 53.7 54.0 54.2
    longitude    (location) float64 6.573 4.949 8.178 ... 7.166 6.657 7.916
Dimensions without coordinates: location
Data variables: (12/14)
    10ws         (time, location) float32 ...
    100ws        (location, time) float64 dask.array<chunksize=(11, 2928), meta=np.ndarray>
    975ws        (time, location) float32 11.87 10.02 13.93 ... 2.31 3.467 5.133
    950ws        (time, location) float32 10.91 8.49 13.75 ... 2.537 4.562 5.13
    t2m          (time, location) float32 ...
    skt          (time, location) float32 ...
    ...           ...
    100alpha     (location, time) float64 dask.array<chunksize=(11, 2928), meta=np.ndarray>
    975wsgrad    (time, location) float64 dask.array<chunksize=(2928, 11), meta=np.ndarray>
    950wsgrad    (time, location) float32 -0.9624 -1.531 ... 1.095 -0.00313
    2mtempgrad   (time, location) float32 -1.651 0.3065 1.16 ... 1.059 0.7938
    950tempgrad  (time, location) float32 -0.7691 0.3322 -1.27 ... -1.823 -2.158
    975tempgrad  (time, location) float32 -1.799 -2.07 -2.153 ... -5.636 -5.293

In [15]:
# === save file ===#
file_path = f'CERRA.nc'
if os.path.exists(file_path):
    os.remove(file_path)
combined_dataset.to_netcdf(file_path)